## Sur l'utilisation des dictionnaires en Python

Tout object "immuable" peut être utilisé comme clef d'un dictionnaire Python

In [1]:
memo = {}
memo[1] = 10
memo['bla'] = 0
memo

{1: 10, 'bla': 0}

Cependant les lists ne sont pas des objets immuables !

In [2]:
l = [1,2,3,4,5]
memo[l] = -1

TypeError: unhashable type: 'list'

En effet, Python associée un hashé à tous les objets immuables, mais pas aux objets dits *"unashable"*

In [3]:
hash('bla')

7418075971660019129

In [4]:
hash(2**1000)

16777216

In [5]:
hash(l)

TypeError: unhashable type: 'list'

Voici une solution pour utiliser des listes comme clefs dans un dictionnaire Python: les convertir en chaînes de caractères

In [6]:
str(l)

'[1, 2, 3, 4, 5]'

In [7]:
l

[1, 2, 3, 4, 5]

In [8]:
memo[str(l)] = 100
memo

{1: 10, 'bla': 0, '[1, 2, 3, 4, 5]': 100}

## Exercice 1

### solution récursive naïve

Avec le meilleur parenthèsage en bonus !

In [9]:
def opt(seq):
    if len(seq) <= 2:
        return 0, 'X'
    m = float("inf")  # Valeur spéciale "infini" (Python-only)
    shape = None
    for i in range(1, len(seq)-1):
        A, shl = opt(seq[:i+1])
        B, shr = opt(seq[i:])
        n = A + B + seq[0]*seq[i]*seq[-1]
        if n < m:
            m = n
            shape = '(%s·%s)' % (shl,shr)
    return m, shape

In [10]:
%time opt([5,10,3,12,5,50,6])

CPU times: user 192 µs, sys: 33 µs, total: 225 µs
Wall time: 229 µs


(2010, '((X·X)·((X·X)·(X·X)))')

### solution récursive avec mémoisation

À partir de maintenant, on donne juste le coût optimal, pour simplicité. La modification pour avoir le parenthèsage est immédiate.

In [11]:
def opt_memo(seq, memo):  # memo est la table de hachage à utiliser pour la mémoisation,
                          # passer un dictionnaire vide lorsqu'on appelle la fonction
    cle = str(seq)
    if cle in memo:
        return memo[cle]
    if len(seq) <= 2:
        return 0
    m = float("inf")  # Valeur spéciale "infini" (Python-only)
    for i in range(1, len(seq)-1):
        n = opt_memo(seq[:i+1], memo) + opt_memo(seq[i:], memo) + seq[0]*seq[i]*seq[-1]
        if n < m:
            m = n
    memo[cle] = m
    return m

In [12]:
%time opt_memo([5,10,3,12,5,50,6], {})

CPU times: user 99 µs, sys: 17 µs, total: 116 µs
Wall time: 120 µs


2010

comparons les performances sur une entrée un peu plus consistante (une partie de la différence est due au fait que `opt` renvoie le parenthésage alors que `opt_memo` ne le fait pas, mais l'impact est négligeable face au gain dû à la mémoisation).

In [13]:
import random

In [14]:
l = random.sample(range(2,100), 15)
l

[50, 22, 63, 35, 75, 78, 67, 2, 84, 73, 29, 3, 53, 64, 97]

In [15]:
%time opt(l)

CPU times: user 1.42 s, sys: 0 ns, total: 1.42 s
Wall time: 1.42 s


(82674, '((X·(X·(X·(X·(X·(X·X))))))·((((((X·X)·X)·X)·X)·X)·X))')

In [16]:
memo = {}
%time opt_memo(l, {})

CPU times: user 1.37 ms, sys: 0 ns, total: 1.37 ms
Wall time: 1.37 ms


82674

### solution dérécursivisée "bottom-up"

À l'analyse de `opt_memo` on reconnaît que la fonction est appellée sur toutes les sous-suites de la suite d'entrée, de longueur $>=2$.

Plutôt que stocker ces valeurs dans une table de hachage, dispendieuse en mémoire, on va utiliser une liste de listes (`bests`, dans le code) occupant le strict minimum de mémoire. Le tableau `bests` est structuré comme ceci : la liste `best[width]` contient les coûts optimaux pour chacune des sous-suites de largeur `width`, de gauche à droite.

Ainsi, pour la suite $(10~5~4~7~8~9~5~3~11)$ en entrée, `bests[4][2]` contiendra le coût optimal pour la troisième suite de longueur $4$, c'est à dire $(4~7~8~9)$.

On ajoute dans le code quelques affichages de debug, pour nous aider à visualiser le tableau `bests` au fur et à mesure qu'il est construit. Lisez la documentation sur le [formatage de chaînes de caractères en Python](https://docs.python.org/3.5/library/string.html#formatspec) pour comprendre la façon dont ces affichages sont construits.

In [17]:
def opt_derec(seq, debug=True):
    if debug:
        print("".join('{:^8}'.format(s) for s in seq))
        print("–" * 8 * len(seq))
    bests = [[], [], [0] * (len(seq) - 1)]
    if debug:
        print(" "*4 + "".join('{:^8}'.format(c) for c in bests[-1]))
    for width in range(3, len(seq) + 1):
        level = []
        for i in range(len(seq) - width + 1):
            cost = min(bests[split + 1][i]
                       + bests[width - split][i + split] 
                       + seq[i]*seq[i+split]*seq[i+width-1]
                       for split in range(1, width - 1))
            level.append(cost)
        bests.append(level)
        if debug:
            print(" "*4*(width-1) + "".join('{:^8}'.format(c) for c in bests[-1]))
    return bests[-1][0]

In [18]:
%time opt_derec(l)

   50      22      63      35      75      78      67      2       84      73      29      3       53      64      97   
––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––
       0       0       0       0       0       0       0       0       0       0       0       0       0       0    
         69300   48510   165375  204750  391950  10452   11256   12264   177828   6351    4611   10176   329024 
             87010   106260  376740  387660  22152   23556   22046   16498   24747   17958   15744   28800  
                 188760  234960  535395  27402   34752   34104   20384   16672   38103   30543   37239  
                     320760  349932  31812   33282   45366   31474   17074   16990   51051   56394  
                         423632  34584   42396   44776   43000   27592   24092   23774   77991  
                             36784   38280   53274   45930   39274   35710   32350   36190  
                       

82674

On voit que le gain devient considéreable lorsqu'on augmente la taille des entrées

In [19]:
l = random.sample(range(2,100), 30)
%time opt_memo(l, {})
%time opt_derec(l, debug=False)

CPU times: user 10.5 ms, sys: 4.17 ms, total: 14.7 ms
Wall time: 14.4 ms
CPU times: user 2.57 ms, sys: 0 ns, total: 2.57 ms
Wall time: 2.57 ms


364080

**Exercice:** avec ce code, il est très simple maintenant d'analyser la complexité de l'algorithme ; donnez-en une estimation asymptotique.